In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

The history saving thread hit an unexpected error (OperationalError('database is locked',)).History will not be written to the database.


In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

atmpro = trp
band = [11]
commitnumber = de5d43e
molecule = {'o3': 'atmpro'}
tsfc = 300


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o o3 band9
  atmpro = mls
  band = 9
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 2e-20
  molecule = o3
  ng_adju = [0, 0]
  ng_refs = [2, 5]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (50, 250)]
  tsfc = 294
  vmax = 3000
  vmin = 1900
  w_diffuse = [(1.55, 1.55), (1.55, 1.55, 1.55, 1.55, 1.8)]
  wgt = [(0.3, 0.4), (0.5, 0.6, 0.7, 0.85, 0.9)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312      1.0 -0.000233                  NaN       NaN
0.000312      NaN       NaN                  1.0 -0.000212
0.000750      2.0 -0.000264                  2.0 -0.000245
0.001052      3.0 -0.000325                  3.0 -0.000293
0.001476      4.0 -0.000387                  4.0 -0.000349
0.002070      5.0 -0.000448                  5.0 -0.000405
0.002904      6.0 -0.000510                  6.0 -0.000461
0.004074      7.0 -0.000571                  7.0 -0.000517
0.005714      8.0 -0.000632                  8.0 -0.000573
0.008015      9.0 -0.000693                  9.0 -0.000630
0.011243     10.0 -0.000755                 10.0 -0.000686
0.015771     11.0 -0.000816                 11.0 -0.000743
0.022122     12.0 -0.000876                 12.0 -0.000800
0.031031     13.0 -0.000937                 13.0 -0.000857
0.043528     14.0 -0.000997                 14.0 -0.000915
0.061057     15.0 -0.001129                 15.0 -0.001046
0.085645     16.0 -0.001351                 16.0 -0.001278
0.120136     17.0 -0.001522                 17.0 -0.001494
0.168516     18.0 -0.001524                 18.0 -0.001595
0.236378     19.0 -0.001239                 19.0 -0.001490
0.331549     20.0 -0.000495                 20.0 -0.001046
0.465100     21.0  0.000960                 21.0 -0.000064
0.652400     22.0  0.003208                 22.0  0.001533
0.915100     23.0  0.004783                 23.0  0.002575
1.283650     24.0  0.004611                 24.0  0.002209
1.800600     25.0  0.003096                 25.0  0.000724
2.525700     26.0 -0.000117                 26.0 -0.002163
3.542800     27.0 -0.004053                 27.0 -0.005584
4.969550     28.0 -0.007577                 28.0 -0.008675
6.970850     29.0 -0.011130                 29.0 -0.012042
9.778100     30.0 -0.015349                 30.0 -0.016239
13.715850    31.0 -0.018433                 31.0 -0.019284
19.239350    32.0 -0.019110                 32.0 -0.019739
26.987250    33.0 -0.018125                 33.0 -0.018326
37.855300    34.0 -0.014319                 34.0 -0.013946
53.100050    35.0 -0.008482                 35.0 -0.007860
73.887500    36.0 -0.003779                 36.0 -0.003343
97.662500    37.0 -0.001503                 37.0 -0.001295
121.437500   38.0 -0.000891                 38.0 -0.000761
145.212500   39.0 -0.000741                 39.0 -0.000634
168.987500   40.0 -0.000640                 40.0 -0.000547
192.762500   41.0 -0.000564                 41.0 -0.000482
216.537500   42.0 -0.000492                 42.0 -0.000420
240.312500   43.0 -0.000430                 43.0 -0.000367
264.087500   44.0 -0.000378                 44.0 -0.000322
287.862500   45.0 -0.000336                 45.0 -0.000286
311.637500   46.0 -0.000303                 46.0 -0.000258
335.412500   47.0 -0.000275                 47.0 -0.000235
359.187500   48.0 -0.000249                 48.0 -0.000213
382.962500   49.0 -0.000226                 49.0 -0.000194
406.737500   50.0 -0.000207                 50.0 -0.000177
430.512500   51.0 -0.000189                 51.0 -0.000162
454.287500   52.0 -0.000171                 52.0 -0.000147
478.062500   53.0 -0.000153                 53.0 -0.000131
501.837500   54.0 -0.000135                 54.0 -0.000116
525.612500   55.0 -0.000116                 55.0 -0.000100
549.387500   56.0 -0.000098                 56.0 -0.000085
573.162500   57.0 -0.000080                 57.0 -0.000069
596.937500   58.0 -0.000061                 58.0 -0.000054
620.712500   59.0 -0.000043                 59.0 -0.000038
644.487500   60.0 -0.000024                 60.0 -0.000023
668.262500   61.0 -0.000006                 61.0 -0.000008
692.037500   62.0  0.000011                 62.0  0.000007
715.812500   63.0  0.000028                 63.0  0.000020
739.587500   64.0  0.000042                 64.0  0.000033
76

# Fluxes by Layer

CRD                            CLIRAD            \
                        flug          fldg     fnetg      flug      fldg   
pressure     level                                                         
0.000000e+00 1     -8.046576  0.000000e+00 -8.046576       NaN       NaN   
1.000000e-08 1           NaN           NaN       NaN -8.051348  0.000002   
6.244000e-04 2     -8.046576  8.531356e-10 -8.046576 -8.051348  0.000002   
8.759000e-04 3     -8.046576  1.249921e-09 -8.046576 -8.051348  0.000002   
1.228600e-03 4     -8.046576  1.964336e-09 -8.046576 -8.051348  0.000002   
1.723400e-03 5     -8.046576  3.206367e-09 -8.046576 -8.051348  0.000002   
2.417400e-03 6     -8.046576  5.311113e-09 -8.046576 -8.051348  0.000002   
3.390900e-03 7     -8.046576  8.811338e-09 -8.046576 -8.051348  0.000002   
4.756500e-03 8     -8.046576  1.454769e-08 -8.046576 -8.051348  0.000002   
6.672000e-03 9     -8.046576  2.383895e-08 -8.046576 -8.051348  0.000002   
9.358900e-03 10    -8.046576  3.874289e-08 -8.046576 -8.051349  0.000002   
1.312780e-02 11    -8.046577  6.246443e-08 -8.046577 -8.051349  0.000002   
1.841450e-02 12    -8.046577  9.996733e-08 -8.046577 -8.051350  0.000002   
2.583020e-02 13    -8.046578  1.589085e-07 -8.046578 -8.051350  0.000002   
3.623230e-02 14    -8.046579  2.511053e-07 -8.046579 -8.051351  0.000002   
5.082340e-02 15    -8.046581  3.963141e-07 -8.046581 -8.051353  0.000002   
7.129060e-02 16    -8.046584  6.752892e-07 -8.046584 -8.051356  0.000003   
1.000000e-01 17    -8.046589  1.365325e-06 -8.046588 -8.051361  0.000003   
1.402710e-01 18    -8.046598  3.139921e-06 -8.046595 -8.051369  0.000005   
1.967600e-01 19    -8.046613  7.509899e-06 -8.046606 -8.051383  0.000008   
2.759970e-01 20    -8.046635  1.789622e-05 -8.046617 -8.051405  0.000015   
3.871000e-01 21    -8.046665  4.185554e-05 -8.046624 -8.051436  0.000033   
5.431000e-01 22    -8.046702  9.578392e-05 -8.046606 -8.051478  0.000074   
7.617000e-01 23    -8.046733  2.101996e-04 -8.046523 -8.051525  0.000161   
1.068500e+00 24    -8.046767  4.183670e-04 -8.046349 -8.051592  0.000321   
1.498800e+00 25    -8.046857  7.430295e-04 -8.046114 -8.051734  0.000575   
2.102400e+00 26    -8.047102  1.209452e-03 -8.045893 -8.052059  0.000952   
2.949000e+00 27    -8.047705  1.801127e-03 -8.045904 -8.052776  0.001452   
4.136600e+00 28    -8.048908  2.433597e-03 -8.046475 -8.054129  0.002019   
5.802500e+00 29    -8.050996  3.026246e-03 -8.047970 -8.056407  0.002586   
8.139200e+00 30    -8.054614  3.563434e-03 -8.051051 -8.060289  0.003134   
1.141700e+01 31    -8.061049  4.039081e-03 -8.057010 -8.067109  0.003649   
1.601470e+01 32    -8.071451  4.400921e-03 -8.067050 -8.078028  0.004063   
2.246400e+01 33    -8.086276  4.625975e-03 -8.081649 -8.093389  0.004343   
3.151050e+01 34    -8.105809  4.735988e-03 -8.101073 -8.113189  0.004503   
4.420010e+01 35    -8.127348  4.750900e-03 -8.122597 -8.134209  0.004557   
6.200000e+01 36    -8.145166  4.684779e-03 -8.140481 -8.150750  0.004524   
8.577500e+01 37    -8.155739  4.615053e-03 -8.151124 -8.160119  0.004476   
1.095500e+02 38    -8.159945  4.586849e-03 -8.155358 -8.163746  0.004456   
1.333250e+02 39    -8.162443  4.577095e-03 -8.157866 -8.165884  0.004450   
1.571000e+02 40    -8.164534  4.580846e-03 -8.159954 -8.167673  0.004455   
1.808750e+02 41    -8.166354  4.596889e-03 -8.161757 -8.169229  0.004469   
2.046500e+02 42    -8.167970  4.624911e-03 -8.163345 -8.170610  0.004492   
2.284250e+02 43    -8.169393  4.664098e-03 -8.164729 -8.171825  0.004524   
2.522000e+02 44    -8.170653  4.713881e-03 -8.165939 -8.172899  0.004565   
2.759750e+02 45    -8.171777  4.773976e-03 -8.167003 -8.173856  0.004614   
2.997500e+02 46    -8.172792  4.844624e-03 -8.167948 -8.174721  0.004672   
3.235250e+02 47    -8.173727  4.926804e-03 -8.168800 -8.175515  0.004739   
3.473000e+02 48    -8.174595  5.021305e-03 -8.169574 -8.176253  0.004815   
3.710750e+02 49    -8.175403  5.128365e-03 -8.170275 -8.176939  0.004902 

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')